In [23]:
#Running python 3.8.12

import pychrono as chrono
from pychrono import irrlicht as chronoirr
import numpy as np
import math as m
import sys
import pandas as pd

chrono.SetChronoDataPath('data')
meshPath = 'C:/Users/sbrege/Documents/coding/mesh/'

In [24]:
#IGNORE
            #Quaternion testing
            # start = chrono.ChQuaternionD(0.71,0.71,0,0)
            # same = chrono.ChQuaternionD()
            # same.Q_from_AngAxis(-m.pi/2, chrono.ChVectorD(0,1,0))

            # right = chrono.ChQuaternionD()
            #right.Q_from_AngAxis(-chrono.CH_C_PI / 2 , chrono.ChVectorD(0, 1, 0))
            
            #self.rat.SetRot(start)

            # rot = self.rat.GetRot()
            # print(rot)
            # ax = self.rat.GetRotAxis()
            # print(ax)
            # ang = self.rat.GetRotAngle()
            # print(ang)

            # A = self.rat.GetA()
            # print(A)

            # coord = self.rat.GetCoord()
            # print(coord)

            # right.Q_from_AngAxis(ang , ax)
            # self.rat.SetRot(right)

            # rot = self.rat.GetRot()
            # print(rot)
            # ax = self.rat.GetRotAxis()
            # print(ax)
            # ang = self.rat.GetRotAngle()
            # print(ang)

            # right.Q_from_AngAxis(ang , -ax)
            # self.rat.SetRot(right)

            # ax = self.rat.GetRotAxis()
            # print(ax)
            # ang = self.rat.GetRotAngle()
            # print(ang)

            #self.rat.SetRot(right)
            #self.rat.SetRot(z2y)

                    #Cage setup, zwalls are along z axis, x walls along x axis
        # if self.arena == 'box':
        #     self.floor = chrono.ChBodyEasyBox(6, 0.5, 6, 3000, True, True, self.material)
        #     self.floor.SetPos(chrono.ChVectorD(0, -0.25, 0))
        #     self.floor.SetBodyFixed(True)
        #     self.floor.SetName('floor')
        #     self.system.Add(self.floor)

        #     self.zwall1 = chrono.ChBodyEasyBox(6,1,0.25, 3000, True, True, self.material)
        #     self.zwall1.SetPos(chrono.ChVectorD(0, 0.5, 3))
        #     self.zwall1.SetBodyFixed(True)
        #     self.zwall1.SetName('wall')
        #     self.system.Add(self.zwall1)

        #     self.zwall2 = chrono.ChBodyEasyBox(6,1,0.25, 3000, True, True, self.material)
        #     self.zwall2.SetPos(chrono.ChVectorD(0, 0.5, -3))
        #     self.zwall2.SetBodyFixed(True)
        #     self.zwall2.SetName('wall')
        #     self.system.Add(self.zwall2)

        #     self.xwall1 = chrono.ChBodyEasyBox(0.25,1,6, 3000, True, True, self.material)
        #     self.xwall1.SetPos(chrono.ChVectorD(3, 0.5, 0))
        #     self.xwall1.SetBodyFixed(True)
        #     self.xwall1.SetName('wall')
        #     self.system.Add(self.xwall1)

        #     self.xwall2 = chrono.ChBodyEasyBox(0.25,1,6, 3000, True, True, self.material)
        #     self.xwall2.SetPos(chrono.ChVectorD(-3, 0.5, 0))
        #     self.xwall2.SetBodyFixed(True)
        #     self.xwall2.SetName('wall')
        #     self.system.Add(self.xwall2)

        #     #Rat mesh setup, currently just a sphere, maybe add better mesh later? Sphere able to roll though
        #     self.rat = chrono.ChBodyEasySphere(0.15, 3000, True, True, self.material)
        #     self.rat.SetPos(chrono.ChVectorD(0, 0.5, 0))
        #     self.rat.SetBodyFixed(False)
        #     self.rat.SetName('rat')
        #     self.system.Add(self.rat)

In [25]:
#Code for collision reports
class RatReportContactCallback(chrono.ReportContactCallback):
     def __init__(self):
          chrono.ReportContactCallback.__init__(self)
          self.bodyA = None
          self.bodyB = None
          self.nameA = None
          self.nameB = None
          self.collideCount = 0
          self.wallCollide = False

     def OnReportContact(self,vA,vB,cA,dist,rad,force,torque,modA,modB):
          self.bodyA = chrono.CastContactableToChBody(modA)
          self.nameA = self.bodyA.GetName()
          self.bodyB = chrono.CastContactableToChBody(modB)
          self.nameB = self.bodyB.GetName() #always seems to be the rat but may not always be true

          if self.nameA == 'wall':
               self.collideCount+=1

          if self.collideCount == 10:
               self.wallCollide = True
               print('wall collide')
               self.collideCount = 0

          #print ('contact: point A=' , vA.z, 'first body:', nameA, 'second body:', nameB)

          return True        # return False to stop reporting contacts



In [32]:
#Useful quaternions
z2x = chrono.ChQuaternionD()
z2x.Q_from_AngAxis(-chrono.CH_C_PI / 2 , chrono.ChVectorD(0, 1, 0))
z2y = chrono.ChQuaternionD()
z2y.Q_from_AngAxis(-chrono.CH_C_PI / 2 , chrono.ChVectorD(0, 1, 0))

#Turns chrono vector into a numpy array
def vec2Array(vector):
    return np.asarray([vector.x, vector.y, vector.z])

def array2Vec(array):
    return chrono.ChVectorD(array[0], array[1], array[2])

#Class that contains the chrono system
class ratSim():
    #init, duh
    def __init__(self, pLen=0.55, pWid=0.15, pHeight=0.15, length=4, width=2, startPos=[0,2], path=[[2,0],[0,-1],[1,0],[0,-1],[1,0]], reportContact=False):

        #Initialising chrono system
        self.system = chrono.ChSystemNSC()
        self.material = chrono.ChMaterialSurfaceNSC()

        chrono.ChCollisionModel.SetDefaultSuggestedEnvelope(0.001)
        chrono.ChCollisionModel.SetDefaultSuggestedMargin(0.001)

        #arena parameters

        self.pLen=pLen
        self.pWid=pWid
        self.pHeight=pHeight
        self.mConst = (pLen-pWid)
        self.length = length
        self.width = width
        self.startPos = startPos
        #path [left/right,down/up] [-1/1, -1/1]
        self.path = np.asarray(path)
        self.reportContact = reportContact

        #Setups the system for simulation
        self.reset()

    def makeKexArena(self, horizontal, vertical):
        pieces = []
        for x in range(horizontal):
            for y in range(vertical):
                #Vertical
                piece = chrono.ChBodyEasyBox(self.pLen, self.pHeight, self.pWid, 3000, True, True, self.material)
                piece.SetPos(chrono.ChVectorD(self.mConst*x + self.mConst/2, -0.2, self.mConst*y))
                piece.SetBodyFixed(True)
                piece.SetName('KexPieceV-' + str(x) + '-' + str(y))
                pieces.append(piece)

                #Horizontal
                piece = chrono.ChBodyEasyBox(self.pWid, self.pHeight, self.pLen, 3000, True, True, self.material)
                piece.SetPos(chrono.ChVectorD(self.mConst*x, -0.2, self.mConst*y + self.mConst/2))
                piece.SetBodyFixed(True)
                piece.SetName('KexPieceH-' + str(x) + '-' + str(y))
                pieces.append(piece)

                #End pieces

                if y == vertical-1:
                    #Vertical
                    piece = chrono.ChBodyEasyBox(self.pLen, self.pHeight, self.pWid, 3000, True, True, self.material)
                    piece.SetPos(chrono.ChVectorD(self.mConst*(x) + self.mConst/2, -0.2, self.mConst*(y+1)))
                    piece.SetBodyFixed(True)
                    piece.SetName('KexPieceV-' + str(x+1) + '-' + str(y))
                    pieces.append(piece)

                if x == horizontal-1:
                    #Horizontal
                    piece = chrono.ChBodyEasyBox(self.pWid, self.pHeight, self.pLen, 3000, True, True, self.material)
                    piece.SetPos(chrono.ChVectorD(self.mConst*(x+1), -0.2, self.mConst*(y) + self.mConst/2))
                    piece.SetBodyFixed(True)
                    piece.SetName('KexPieceH-' + str(x) + '-' + str(y+1))
                    pieces.append(piece)

        #Add pieces to system
        for piece in pieces:
            self.system.Add(piece)

    #Clears and adds elements to system
    def reset(self):
        #Clears system, duh
        self.system.Clear()
        self.system.Set_G_acc(chrono.ChVectorD(0,0,0))

        self.makeKexArena(self.length, self.width)

        #rat mesh

        self.rat = chrono.ChBodyEasySphere(0.05, 3000, True, True, self.material)
        self.rat.SetBodyFixed(False)
        self.rat.SetCollide(False)
        self.rat.SetName('rat')
        #self.rat.AddAsset(chrono.ChColorAsset(1,0.4,0.4))
        self.rat.SetPos(chrono.ChVectorD(self.startPos[0]*self.mConst, -0.1, self.startPos[1]*self.mConst))
        self.system.Add(self.rat)

        #Joint for actuator and rat

        self.joint = chrono.ChBodyEasyBox(0.05,0.05,0.05, 3000, False, False, self.material)
        #actual position unimportant, can be ignored
        self.joint.SetPos(chrono.ChVectorD(0, -0.1, 0.6))
        self.joint.SetName('joint')
        self.joint.SetBodyFixed(True)
        self.system.Add(self.joint)

        #actuator to move rat

        self.actuator = chrono.ChLinkLockLock()
        self.actuator.Initialize(self.rat, self.joint, chrono.CSYSNORM)
        self.system.Add(self.actuator)

        self.actuator.SetMotion_X(chrono.ChFunction_Const(0))
        self.actuator.SetMotion_Z(chrono.ChFunction_Const(0))

        if self.reportContact:
            self.rep = RatReportContactCallback()

    #Runs IRR simulation, which is visualized
    def runIRR(self):
        #Boring setup, self explanatory
        self.application = chronoirr.ChIrrApp(self.system, "Grid cell sim", chronoirr.dimension2du(1224, 900), False)
        self.application.SetTryRealtime(True)

        self.application.AddTypicalSky()
        self.application.AddTypicalLights()
        self.application.AddTypicalCamera(chronoirr.vector3df(2, 1, -0.5))
        self.application.AddLightWithShadow(chronoirr.vector3df(20.0, 35.0, -25.0), chronoirr.vector3df(0, 0, 0), 55, 20, 55, 35, 512,
                                    chronoirr.SColorf(0.6, 0.8, 1.0))

        self.application.AssetBindAll()
        self.application.AssetUpdateAll()
        self.application.AddShadowAll()

        self.solver = chrono.ChSolverPSOR()
        self.solver.SetMaxIterations(50)
        self.system.SetSolver(self.solver)

        #Timestep 
        self.application.SetTimestep(0.01)
        self.application.SetTryRealtime(True)

        #Timestep counter
        ts = 0
        timeout = 0

        #helper variables
        loop = 0

        #previous action in sequence
        pAction = np.asarray([0,0])

        #contains all previous actions for movement
        pCumulative = np.asarray([0,0])

        nextMove = True

        #Behavioral recordings
        velocityArray = []

        #Runs the IRR sim one timestep at a time, simulation will stop when pop-up is closed or application is stopped inside the loop
        while self.application.GetDevice().run() :
            self.application.BeginScene(True, True, chronoirr.SColor(255, 140, 161, 192))

            self.application.DrawAll()

            t = self.system.GetChTime()

            if self.reportContact:
                self.system.GetContactContainer().ReportAllContacts(self.rep)

            if loop < len(self.path):
                cVel = vec2Array(self.rat.GetPos_dt())
                velocityArray.append(cVel)
                cAction = np.asarray(self.path[loop])
                
                
                if nextMove:

                    #Vertical
                    self.actuator.SetMotion_Z(chrono.ChFunction_Const((cAction[1]+pCumulative[1])*self.mConst))
                    
                    #Horizontal
                    self.actuator.SetMotion_X(chrono.ChFunction_Const((cAction[0]+pCumulative[0])*self.mConst))

                    pAction = cAction
                    pCumulative += cAction
                    nextMove = False
                    timeout = 0

                self.application.DoStep()

                ts+=1
                timeout += 1

                #Determines when it has finished a move
                if timeout >= 7:
                    if np.sum(np.abs(cVel)) < 0.2:
                        nextMove = True
                        loop += 1
                        timeout = 0
            
            self.application.EndScene()

        return velocityArray

        #Closes down IRR application fully and resets so it can be run again.
        self.application.GetDevice().closeDevice()
        self.reset()

        


In [33]:
test = ratSim()
velocity_data = pd.DataFrame(test.runIRR(), columns=['x','y','z'])

In [34]:
velocity_data.to_csv('velocity_data.csv')